# Forward Rate Agreements Cash Settlement

This notebook demonstrates how to access and use the functionalities of **Forward Rate Agreements (FRA) cash settlement** within **LSEG Financial Analytics SDK**. 

The notebook outlines a simple method of cash settlement calculation for a set of FRA instruments.

**You will be able to:**
- Define the FRA instruments by tailoring its features
- Estimate spread and cash settlement
- Understand how FRA valuation is calculated

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.forward_rate_agreement` - for Forward Rate Agreement instruments definitions and analytics

In [1]:
from lseg_analytics.pricing.instruments import forward_rate_agreement as fra
import datetime as dt
import json

## Data Preparation

To define a FRA instrument you need to follow a structured 3-step process:
1. **FRA Definition** - Specify the FRA instrument definition
2. **FRA Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional

We take here an example of JPY TONAR FRA with start tenor 5M and end tenor 8M priced in historical time (valuation on 2025-07-21).

In [2]:
# Create FRA defintion object
fra_definition = fra.ForwardRateAgreementDefinition(
    start_tenor="5M", # mandatory
    fixed_rate_percent=4,
    end_tenor="8M", # mandatory
    notional_ccy="JPY", # mandatory
    index_name="TONAR",
    notional_amount=1000000
)

# Create FRA instrument defintion object
fra_instrument = fra.ForwardRateAgreementDefinitionInstrument(
    definition = fra_definition
)

# Create FRA pricing parameters object - optional
fra_parameters = fra.ForwardRateAgreementPricingParameters(
    valuation_date  = dt.datetime.strptime("2025-07-21", "%Y-%m-%d")
)

## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

fra_response = fra.price(
    definitions = [fra_instrument], 
    pricing_preferences = fra_parameters
)

## Results Display

The results showed here are focused on term FRA cash settlement and valuation construction. We then give all necessary outputs: 
- description 
- cash-flows
- pricing analysis
- valuation

In [4]:
# Access the description object
description = fra_response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentDescription": "5Mx8M",
    "tradeDate": "2025-07-21T00:00:00Z",
    "spotDate": "2025-07-23T00:00:00Z",
    "fixingDate": "2025-12-19T00:00:00Z",
    "startDate": "2025-12-23T00:00:00Z",
    "endDate": "2026-03-23T00:00:00Z",
    "tenor": "3M",
    "calendar": "JAP_FI",
    "notionalCcy": "JPY",
    "notionalAmount": 1000000.0,
    "interestCalculationMethod": "Dcb_Actual_360",
    "indexName": "TONAR",
    "indexTenor": "3M",
    "priceSide": "Mid",
    "forwardCurveName": "JPY LIBOR 3M",
    "discountCurveName": "JPY LIBOR 3M"
}


In [5]:
# Access to cash settlement
cash_settlement = fra_response.data.analytics[0].cashflows.cash_flows[0]["payments"][0]
print(json.dumps(cash_settlement, indent=4))

{
    "date": "2025-12-23",
    "amount": -8310.00460684849,
    "currency": "JPY",
    "event": "CashSettlement",
    "type": "Float",
    "occurence": "Projected"
}


In [6]:
# Access the pricing analysis object
pricing_analysis = fra_response.data.analytics[0].pricing_analysis
print(json.dumps(pricing_analysis.as_dict(), indent=4))

{
    "marketDataDate": "2025-07-21T00:00:00Z",
    "valuationDate": "2025-07-21T00:00:00Z",
    "discountFactor": 0.996058981898809,
    "fixedRatePercent": 4.0,
    "parRatePercent": 0.675998157260604,
    "spreadBp": 332.40018427394
}


In [7]:
# Access the valuation object
valuation = fra_response.data.analytics[0].valuation
print(json.dumps(valuation.as_dict(), indent=4))

{
    "marketValueInDealCcy": -8277.25472827192
}


Before building the cash settlement itself, we first show that the *spreadBp* field is built according to the following formula : $$SpreadBp = 100\cdot\left(FixedRatePercent-ParRatePercent\right)$$

In [8]:
print("RatePercent difference computed x 100:", 100 * (pricing_analysis.fixed_rate_percent - pricing_analysis.par_rate_percent))
print("SpreadBp returned:", pricing_analysis.spread_bp)
print("Difference:", 100 * (pricing_analysis.fixed_rate_percent - pricing_analysis.par_rate_percent) - pricing_analysis.spread_bp)

RatePercent difference computed x 100: 332.4001842739396
SpreadBp returned: 332.40018427394
Difference: -3.979039320256561e-13


The cash settlement is derived according to the following formula:
$$CashSettlement :=Notional\cdot\frac{ParRatePercent-FixedRatePercent}{100}\cdot DayCountFraction\left(StartDate, EndDate\right)$$

In this use-case, day count convention is Act/360, meaning that it is the number of days (here 90) divided by 360. We apply below this formula to explain its construction. 

In [9]:
cash_settlement_amount = cash_settlement["amount"]
nb_days = (description.end_date - description.start_date).days
computed_cash_settlement_amount = description.notional_amount * ((pricing_analysis.par_rate_percent - pricing_analysis.fixed_rate_percent) / 100) * (nb_days / 360)
print("Day count convention:", description.interest_calculation_method)
print("Computed cash settlement:", computed_cash_settlement_amount)
print("Cash settlement returned by pricer:", cash_settlement_amount)
print("Difference:", computed_cash_settlement_amount - cash_settlement_amount)

Day count convention: Dcb_Actual_360
Computed cash settlement: -8310.004606848492
Cash settlement returned by pricer: -8310.00460684849
Difference: -1.8189894035458565e-12


Finally, note that FRA valuation and the cash settlement are linked by the relation
$$Valuation_{FRA}=DiscountFactor\cdot CashSettlement$$
which is showed by this last code block:

In [10]:
computed_fra_valuation = computed_cash_settlement_amount * pricing_analysis.discount_factor
print("Computed valuation:", computed_fra_valuation)
print("Valuation returned by pricer:", valuation.market_value_in_deal_ccy)
print("Difference:", computed_fra_valuation - valuation.market_value_in_deal_ccy)

Computed valuation: -8277.25472827192
Valuation returned by pricer: -8277.25472827192
Difference: 0.0
